In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

In [2]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [3]:
X_train = train_data.drop(columns= 'SalePrice')
y_train = train_data['SalePrice']
X_test = test_data.copy()

In [4]:
y_train = np.log1p(train_data['SalePrice'])

In [5]:
missing_values_Percentage = train_data.isnull().mean() * 100
columns_to_drop = missing_values_Percentage[missing_values_Percentage > 40].index
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)
print(f"Dropped columns: {list(columns_to_drop)}")

Dropped columns: ['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


کامیت

In [6]:
num_imputer = SimpleImputer(strategy='median')

numerical_X = X_train.select_dtypes(include=[np.number]).columns

X_train[numerical_X] = num_imputer.fit_transform(X_train[numerical_X])
X_test[numerical_X] = num_imputer.transform(X_test[numerical_X])

In [7]:
cat_imputer = SimpleImputer(strategy='most_frequent')

categorical_X = X_train.select_dtypes(include=['object']).columns

X_train[categorical_X] = cat_imputer.fit_transform(X_train[categorical_X])
X_test[categorical_X] = cat_imputer.transform(X_test[categorical_X])